# Symbolic neural networks for cognitive capacities
[Paper](http://reason.cs.uiuc.edu/tsvi/BICA_93_Main.pdf)

## Example of a more complex symbolic learning, recognition and recall

Take 3 men (Adam, Baker, Charlie) and 3 women (Diane, Edna, Fay) in a group which may hold 9 people.

Make a matrix of features. Gender (1=male, 2=female), name_adam? (1=yes, 0=no), name_baker?, ... etc.

In [1]:
import numpy as np
exp = np.zeros((9, 10))
exp_labels = []
print(exp)
print(exp.shape)

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
(9, 10)


Define **Instance** and **Property** learning functions.

In [2]:
def softmax_2(x):
    e_x = np.exp(x - np.max(x))
    out = e_x / e_x.sum()
    return out

def vectorize(instance, exp_labels):
    instance_vector = np.zeros(exp.shape[1])
    for key in instance.keys():
        # add key to labels, if needed
        if not key in exp_labels:
            exp_labels.append(key)
        key_offset = exp_labels.index(key)
        instance_vector.put(key_offset, instance[key])
    return np.matrix(instance_vector), exp_labels

def learn_instance(instance_list, exp, exp_labels):
    for instance in instance_list:
        instance_vector, exp_labels = vectorize(instance, exp_labels)
        # find first 0 row
        for i in range(exp.shape[0]):
            if not exp[i].any():
                exp[i] = instance_vector
                break
    return exp, exp_labels

def learn_property(instance_list, exp, exp_labels):
    for instance in instance_list:
        instance_vector, exp_labels = vectorize(instance, exp_labels)
        # find first row with 1 unmatched element
        for i in range(exp.shape[0]):
            if not exp[i].any():
                break
            else:
                difference = instance_vector - exp[i]
                if np.count_nonzero(difference) == 1:
                    difference_index = np.nonzero(difference)[1][0]
                    exp.put(i * exp.shape[1] + difference_index, difference.item((0, difference_index)))
                break   
    return exp, exp_labels

Learn first instance

In [3]:
Adam = {"name_adam": 1}
exp, exp_labels = learn_instance([Adam,], exp, exp_labels)
print(exp)
print(exp_labels)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
['name_adam']


Learn first property

In [4]:
Adam = {"name_adam": 1, "gender": 1}
exp, exp_labels = learn_property([Adam,], exp, exp_labels)
print(exp)
print(exp_labels)

[[ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
['name_adam', 'gender']


In [5]:
Baker  = {"gender": 1, "name_baker": 1}
Charlie = {"gender": 1, "name_charlie": 1}
Diane = {"gender": 2, "name_diane": 1}
Edna = {"gender": 2, "name_edna": 1}
Fay = {"gender": 2, "name_fay": 1}
exp, exp_labels = learn_instance([Baker, Charlie, Diane, Edna, Fay], exp, exp_labels)
print(exp)
print(exp_labels)

[[ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
['name_adam', 'gender', 'name_baker', 'name_charlie', 'name_diane', 'name_edna', 'name_fay']


Check that each instance is recognised

In [6]:
pinv = np.linalg.pinv(exp)
pinv_transpose = pinv.transpose()
print(pinv_transpose)

[[ 0.9375  0.0625 -0.0625 -0.0625 -0.125  -0.125  -0.125   0.      0.      0.    ]
 [-0.0625  0.0625  0.9375 -0.0625 -0.125  -0.125  -0.125   0.      0.      0.    ]
 [-0.0625  0.0625 -0.0625  0.9375 -0.125  -0.125  -0.125   0.      0.      0.    ]
 [-0.125   0.125  -0.125  -0.125   0.75   -0.25   -0.25    0.      0.      0.    ]
 [-0.125   0.125  -0.125  -0.125  -0.25    0.75   -0.25    0.      0.      0.    ]
 [-0.125   0.125  -0.125  -0.125  -0.25   -0.25    0.75    0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]]


In [7]:
for index in range(exp.shape[0]):
    prediction = pinv_transpose * np.matrix(exp[index]).T
    print(index + 1, round(prediction.item( (index, 0)), 2))

1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 0.0
8 0.0
9 0.0


# Categories

Define two categories, male and female.  These are the SUM of the observations in each set.

In [8]:
def make_male_and_female_categories(exp):
    male = np.matrix(np.zeros(exp.shape[1]))
    female = np.matrix(np.zeros(exp.shape[1]))
    for index in range(exp.shape[0]):
        if exp[index].any():
            if exp.item( (index, exp_labels.index('gender')) ) == 1:
                male += exp[index, :]
            else:
                female += exp[index, :]
    return male, female
male, female = make_male_and_female_categories(exp)
print("male  ", male)
print("female", female)

male   [[ 1.  3.  1.  1.  0.  0.  0.  0.  0.  0.]]
female [[ 0.  6.  0.  0.  1.  1.  1.  0.  0.  0.]]


Check predictions of each category

In [9]:
prediction_male = pinv_transpose * male.T
prediction_male.round(2)

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])

In [10]:
prediction_female = pinv_transpose * female.T
prediction_female.round(2)

array([[-0.],
       [-0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.]])

Explore confounding cases

In [11]:
female_with_male_name = {"gender": 2, "name_adam": 1}
female_with_male_name_vector, exp_labels = vectorize(female_with_male_name, exp_labels)
female_with_male_name_vector

matrix([[ 1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [12]:
prediction_male_names = pinv_transpose * female_with_male_name_vector.T
prediction_male_names.round(2)

array([[ 1.06],
       [ 0.06],
       [ 0.06],
       [ 0.13],
       [ 0.13],
       [ 0.13],
       [ 0.  ],
       [ 0.  ],
       [ 0.  ]])

In [13]:
male_with_female_name = {"gender": 1, "name_fay": 1}
male_with_female_name_vector, exp_labels = vectorize(male_with_female_name, exp_labels)
prediction_female_names = pinv_transpose * male_with_female_name_vector.T
prediction_female_names.round(2)

array([[-0.06],
       [-0.06],
       [-0.06],
       [-0.12],
       [-0.12],
       [ 0.88],
       [ 0.  ],
       [ 0.  ],
       [ 0.  ]])

## Add a new person, George

In [14]:
George = {"gender": 1, "name_george": 1}
exp, exp_labels = learn_instance([George,], exp, exp_labels)
print(exp)
print(exp_labels)

[[ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
['name_adam', 'gender', 'name_baker', 'name_charlie', 'name_diane', 'name_edna', 'name_fay', 'name_george']


Check that George is recognised

In [15]:
pinv = np.linalg.pinv(exp)
pinv_transpose = pinv.transpose()
predict_george = pinv_transpose * vectorize(George, exp_labels)[0].T
predict_george.round(2)

array([[-0.],
       [ 0.],
       [-0.],
       [ 0.],
       [-0.],
       [-0.],
       [ 1.],
       [ 0.],
       [ 0.]])

Is George male?

In [16]:
exp[exp[:, exp_labels.index('name_george')] == 1, exp_labels.index('gender')][0]

1.0

Is there a male named George?

In [17]:
males = exp[exp[:, exp_labels.index('gender')] == 1]
males[males[:, exp_labels.index('name_george')] == 1]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]])

Is there a male named Fay?

In [18]:
males = exp[exp[:, exp_labels.index('gender')] == 1]
males[males[:, exp_labels.index('name_fay')] == 1]

array([], shape=(0, 10), dtype=float64)

## Add _female_ George (like [George Eliot](https://en.wikipedia.org/wiki/George_Eliot))

In [19]:
George2 = {"gender": 2, "name_george": 1}
exp, exp_labels = learn_instance([George2,], exp, exp_labels)
print(exp)
print(exp_labels)

[[ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  2.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
['name_adam', 'gender', 'name_baker', 'name_charlie', 'name_diane', 'name_edna', 'name_fay', 'name_george']


Check that female George is recognised

In [20]:
pinv = np.linalg.pinv(exp)
pinv_transpose = pinv.transpose()
predict_george_f = pinv_transpose * vectorize(George2, exp_labels)[0].T
predict_george_f.round()

matrix([[-0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [-0.],
        [ 1.],
        [ 0.]])

Is George male, female -- or both?

In [21]:
exp[exp[:, exp_labels.index('name_george')] == 1, exp_labels.index('gender')]

array([ 1.,  2.])

Update to categories

In [22]:
male, female = make_male_and_female_categories(exp)
print("male  ", male)
print("female", female)
prediction_male = pinv_transpose * male.T
prediction_male.round(2)

male   [[ 1.  4.  1.  1.  0.  0.  0.  1.  0.  0.]]
female [[ 0.  8.  0.  0.  1.  1.  1.  1.  0.  0.]]


array([[ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.]])

In [23]:
prediction_female = pinv_transpose * female.T
prediction_female.round(2)

array([[-0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-0.],
       [ 1.],
       [ 0.]])

Make new category: Georges

In [24]:
def make_georges_category(exp, exp_labels):
    georges = np.matrix(np.zeros(exp.shape[1]))
    for index in range(exp.shape[0]):
        if exp[index].any():
            if exp.item( (index, exp_labels.index('name_george')) ) == 1:
                georges += exp[index, :]
    return georges
georges = make_georges_category(exp, exp_labels)
georges

matrix([[ 0.,  3.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.]])

In [25]:
prediction_georges = pinv_transpose * georges.T
prediction_georges.round(2)

array([[-0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.]])

# Generalizations

In [26]:
new_exp = male
new_exp = np.concatenate((new_exp, female), axis=0)
new_exp = np.concatenate((new_exp, georges), axis=0)
new_exp

matrix([[ 1.,  4.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  8.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.],
        [ 0.,  3.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.]])

In [27]:
pinv = np.linalg.pinv(new_exp)
pinv_transpose = pinv.transpose()
for index in range(new_exp.shape[0]):
    prediction = pinv_transpose * np.matrix(new_exp[index, :]).T
    print(index + 1, round(prediction.item( (index, 0)), 2))

1 1.0
2 1.0
3 1.0


What do _male_ and _georges_ have in common?

In [28]:
male

matrix([[ 1.,  4.,  1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.]])

In [29]:
georges

matrix([[ 0.,  3.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.]])

In [30]:
common = np.multiply(male, georges)
common

matrix([[  0.,  12.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,   0.]])

In [31]:
for index in range(common.shape[1]):
    if common.item( (0, index) ):
        print(exp_labels[index])

gender
name_george


And _female_ and _georges_?

In [32]:
def in_common(category_a, category_b):
    common = np.multiply(category_a, category_b)
    for index in range(common.shape[1]):
        if common.item( (0, index) ):
            print(exp_labels[index])
in_common(female, georges)

gender
name_george


And _male_ and _female_?

In [33]:
in_common(male, female)

gender
name_george
